# ETL Pipeline

## Extract - Tratamento de Campos

In [1]:
import pandas as pd
import pandera as pa
import regex as re

In [2]:
df = pd.read_csv("bolsistas_iniciacao_cientifica.csv", sep=';', encoding='UTF-8')

In [3]:
df.head(10)

,id_discente,matricula,discente,titulo,codigo_projeto,id_projeto_pesquisa,ano,id_orientador,orientador,categoria,tipo_de_bolsa,linha_pesquisa,id_grupo_pesquisa,grupo_pesquisa,cota,inicio,fim,id_unidade,unidade,status
0,739553,20170093636,ABDA ROCHA PINHEIRO,Pesquisando práticas musicais no contexto de N...,PVH15882-2018,129546741,2018,5751926,JEAN JOUBERT FREITAS MENDES,Iniciação Científica (IC),VOLUNTÁRIO,Processos sociais e práticas musicais na conte...,135380677.0,GPMuC - Grupo de Pesquisa Música em Múltiplos ...,2019-2020,2019/08/05 00:00:00.000000000,2020/07/31 00:00:00.000000000,284,ESCOLA DE MÚSICA - EMUFRN,PENDENTE DE RELATORIO
1,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avança...,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Tecnológica (IT),PIBITI UFRN,Fotoquímica e Equilíbrio de Fases,77.0,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020/08/10 00:00:00.000000000,2020/09/03 00:00:00.000000000,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
2,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avança...,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Científica (IC),PIBIC AF CNPq,Fotoquímica e Equilíbrio de Fases,77.0,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020/09/03 00:00:00.000000000,2021/08/31 00:00:00.000000000,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
3,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estr...,PVE334-2007,1179265,2007,64653,MARIANA BALDI,Iniciação Científica (IC),PIBIC,Organizações e comportamento organizacional;;;,38.0,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2007-2008,2007/08/01 00:00:00.000000000,2008/07/31 00:00:00.000000000,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
4,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estr...,PVE334-2006,41258,2006,64653,MARIANA BALDI,Iniciação Científica (IC),PROPESQ,Organizações e comportamento organizacional;;;,38.0,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2006-2007,2006/08/01 00:00:00.000000000,2007/07/31 00:00:00.000000000,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
5,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,ELEMENTOS DO COMPORTAMENTO ORGANIZACIONAL:Tipo...,PVE513-2008,5487590,2008,23198,WASHINGTON JOSE DE SOUSA,Iniciação Científica (IC),PIBIC,Gestão de Organizações Sociais;Trabalho Volunt...,43.0,GRUPO DE ESTUDOS E PESQUISAS EM GESTÃO INSTITU...,2008-2009,2008/11/06 00:00:00.000000000,2009/07/31 00:00:00.000000000,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
6,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,Motivos de Adesão e Filiação ao Trabalho Volun...,PVE3275-2009,11687003,2009,23198,WASHINGTON JOSE DE SOUSA,Iniciação Científica (IC),PIBIC,Análise e Avaliação de Políticas Públicas;;;,NaN,NaN,2009-2010,2009/08/01 00:00:00.000000000,2009/12/25 00:00:00.000000000,161,DEPARTAMENTO DE CIÊNCIAS ADMINISTRATIVAS - DEP...,FINALIZADO
7,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,Geração e disseminação da inovação na carcinic...,PVE2463-2008,5554243,2008,64653,MARIANA BALDI,Iniciação Científica (IC),PIBIC,Organizações e comportamento organizacional,38.0,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2008-2009,2008/08/25 00:00:00.000000000,2008/11/06 13:35:24.877000000,161,DEPARTAMENTO DE CIÊNCIAS ADMINISTRATIVAS - DEP...,FINALIZADO
8,545009,2011049415,ABEL LUIS DA SILVA NETO,Estudo de Antenas com Reflectarrays e suas Apl...,PVG6992-2013,97873251,2013,5754609,ADAILDO GOMES D ASSUNCAO,Iniciação Tecnológica (IT),PIBIT,"ANTENAS, PROPAGAÇÃO E COMUNICAÇÕES SEM FIO",16.0,GRUPO DE MICROONDAS E ANTENAS,2013-2014,2013/08/14 00:00:00.000000000,2014/06/26 00:00:00.000000000,5204,DEPARTAMENTO DE ENGENHARIA DE COMUNICACOES - DCO,PENDENTE DE RELATORIO
9,74553,200234893,ABESON CORINGA DE MOURA,A HISTÓRIA DO MOVIMENTO DA MATEMÁTICA MODERNA ...,PVB758-2004,37402,2004,24417,ARLETE DE JESUS BRITO,In

In [4]:
df.shape

(40502, 20)

#### Ele se tornou float por ter nulos, porém neste tipo de dados temos que ter decimais, mas no banco ele é PK inteiro.

In [5]:
# É necessário manter os NULOS para facilitar o processo de carga no Oracle
df1 = df['id_grupo_pesquisa'].astype(str)
df.id_grupo_pesquisa = df1.apply(lambda x: x.split('.')[0]).replace('nan', pd.NA)

### Tranformar a data Inicio e Fim, para apenas data, sem a Hora.

In [6]:
df['inicio'] = pd.to_datetime(df['inicio']).dt.normalize()

In [7]:
df['fim'] = pd.to_datetime(df['fim']).dt.normalize()

In [8]:
df.head(10)

,id_discente,matricula,discente,titulo,codigo_projeto,id_projeto_pesquisa,ano,id_orientador,orientador,categoria,tipo_de_bolsa,linha_pesquisa,id_grupo_pesquisa,grupo_pesquisa,cota,inicio,fim,id_unidade,unidade,status
0,739553,20170093636,ABDA ROCHA PINHEIRO,Pesquisando práticas musicais no contexto de N...,PVH15882-2018,129546741,2018,5751926,JEAN JOUBERT FREITAS MENDES,Iniciação Científica (IC),VOLUNTÁRIO,Processos sociais e práticas musicais na conte...,135380677,GPMuC - Grupo de Pesquisa Música em Múltiplos ...,2019-2020,2019-08-05,2020-07-31,284,ESCOLA DE MÚSICA - EMUFRN,PENDENTE DE RELATORIO
1,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avança...,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Tecnológica (IT),PIBITI UFRN,Fotoquímica e Equilíbrio de Fases,77,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020-08-10,2020-09-03,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
2,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avança...,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Científica (IC),PIBIC AF CNPq,Fotoquímica e Equilíbrio de Fases,77,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020-09-03,2021-08-31,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
3,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estr...,PVE334-2007,1179265,2007,64653,MARIANA BALDI,Iniciação Científica (IC),PIBIC,Organizações e comportamento organizacional;;;,38,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2007-2008,2007-08-01,2008-07-31,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
4,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estr...,PVE334-2006,41258,2006,64653,MARIANA BALDI,Iniciação Científica (IC),PROPESQ,Organizações e comportamento organizacional;;;,38,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2006-2007,2006-08-01,2007-07-31,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
5,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,ELEMENTOS DO COMPORTAMENTO ORGANIZACIONAL:Tipo...,PVE513-2008,5487590,2008,23198,WASHINGTON JOSE DE SOUSA,Iniciação Científica (IC),PIBIC,Gestão de Organizações Sociais;Trabalho Volunt...,43,GRUPO DE ESTUDOS E PESQUISAS EM GESTÃO INSTITU...,2008-2009,2008-11-06,2009-07-31,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
6,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,Motivos de Adesão e Filiação ao Trabalho Volun...,PVE3275-2009,11687003,2009,23198,WASHINGTON JOSE DE SOUSA,Iniciação Científica (IC),PIBIC,Análise e Avaliação de Políticas Públicas;;;,<NA>,NaN,2009-2010,2009-08-01,2009-12-25,161,DEPARTAMENTO DE CIÊNCIAS ADMINISTRATIVAS - DEP...,FINALIZADO
7,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,Geração e disseminação da inovação na carcinic...,PVE2463-2008,5554243,2008,64653,MARIANA BALDI,Iniciação Científica (IC),PIBIC,Organizações e comportamento organizacional,38,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2008-2009,2008-08-25,2008-11-06,161,DEPARTAMENTO DE CIÊNCIAS ADMINISTRATIVAS - DEP...,FINALIZADO
8,545009,2011049415,ABEL LUIS DA SILVA NETO,Estudo de Antenas com Reflectarrays e suas Apl...,PVG6992-2013,97873251,2013,5754609,ADAILDO GOMES D ASSUNCAO,Iniciação Tecnológica (IT),PIBIT,"ANTENAS, PROPAGAÇÃO E COMUNICAÇÕES SEM FIO",16,GRUPO DE MICROONDAS E ANTENAS,2013-2014,2013-08-14,2014-06-26,5204,DEPARTAMENTO DE ENGENHARIA DE COMUNICACOES - DCO,PENDENTE DE RELATORIO
9,74553,200234893,ABESON CORINGA DE MOURA,A HISTÓRIA DO MOVIMENTO DA MATEMÁTICA MODERNA ...,PVB758-2004,37402,2004,24417,ARLETE DE JESUS BRITO,Iniciação Científica (IC),VOLUNTÁRIO,História da Educação no Brasil;História da Ciê...,112,MATEMÁTICA E CULTURA,2004-2005,2004-08-01,2005-07-31,439,CENTRO DE CIÊNCIAS EXATAS E DA TERRA - CCET,FINALIZADO


In [9]:
df.dtypes

id_discente                     int64
matricula                       int64
discente                       object
titulo                         object
codigo_projeto                 object
id_projeto_pesquisa             int64
ano                             int64
id_orientador                   int64
orientador                     object
categoria                      object
tipo_de_bolsa                  object
linha_pesquisa                 object
id_grupo_pesquisa              object
grupo_pesquisa                 object
cota                           object
inicio                 datetime64[ns]
fim                    datetime64[ns]
id_unidade                      int64
unidade                        object
status                         object
dtype: object

In [10]:
df.isnull().any()

id_discente            False
matricula              False
discente               False
titulo                  True
codigo_projeto         False
id_projeto_pesquisa    False
ano                    False
id_orientador          False
orientador             False
categoria              False
tipo_de_bolsa          False
linha_pesquisa          True
id_grupo_pesquisa       True
grupo_pesquisa          True
cota                   False
inicio                  True
fim                     True
id_unidade             False
unidade                False
status                 False
dtype: bool

#### Com o Pandera, é possível realizar um tratamento de atributos de forma mais prática

In [11]:
schema = pa.DataFrameSchema(
    columns = {
        "id_discente":pa.Column(pa.Int),
        "matricula":pa.Column(pa.Int),
        "discente":pa.Column(pa.String),
        "titulo":pa.Column(pa.Object, nullable=True),
        "codigo_projeto":pa.Column(pa.String),
        "id_projeto_pesquisa":pa.Column(pa.Int),
        "ano":pa.Column(pa.Int),
        "id_orientador":pa.Column(pa.Int),
        "orientador":pa.Column(pa.String),
        "categoria":pa.Column(pa.String),
        "tipo_de_bolsa":pa.Column(pa.String),
        "linha_pesquisa":pa.Column(pa.String, nullable=True),
        "id_grupo_pesquisa":pa.Column(pa.Object, nullable=True),
        "grupo_pesquisa":pa.Column(pa.String, nullable=True),
        "cota":pa.Column(pa.String),
        "inicio":pa.Column(pa.DateTime, nullable=True),
        "fim":pa.Column(pa.DateTime, nullable=True),
        "id_unidade":pa.Column(pa.Int),
        "unidade":pa.Column(pa.String),
        "status":pa.Column(pa.String)
    }
)

In [12]:
schema.validate(df, inplace=True)

,id_discente,matricula,discente,titulo,codigo_projeto,id_projeto_pesquisa,ano,id_orientador,orientador,categoria,tipo_de_bolsa,linha_pesquisa,id_grupo_pesquisa,grupo_pesquisa,cota,inicio,fim,id_unidade,unidade,status
0,739553,20170093636,ABDA ROCHA PINHEIRO,Pesquisando práticas musicais no contexto de N...,PVH15882-2018,129546741,2018,5751926,JEAN JOUBERT FREITAS MENDES,Iniciação Científica (IC),VOLUNTÁRIO,Processos sociais e práticas musicais na conte...,135380677,GPMuC - Grupo de Pesquisa Música em Múltiplos ...,2019-2020,2019-08-05,2020-07-31,284,ESCOLA DE MÚSICA - EMUFRN,PENDENTE DE RELATORIO
1,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avança...,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Tecnológica (IT),PIBITI UFRN,Fotoquímica e Equilíbrio de Fases,77,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020-08-10,2020-09-03,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
2,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avança...,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Científica (IC),PIBIC AF CNPq,Fotoquímica e Equilíbrio de Fases,77,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020-09-03,2021-08-31,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
3,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estr...,PVE334-2007,1179265,2007,64653,MARIANA BALDI,Iniciação Científica (IC),PIBIC,Organizações e comportamento organizacional;;;,38,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2007-2008,2007-08-01,2008-07-31,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
4,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estr...,PVE334-2006,41258,2006,64653,MARIANA BALDI,Iniciação Científica (IC),PROPESQ,Organizações e comportamento organizacional;;;,38,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E P...,2006-2007,2006-08-01,2007-07-31,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40497,711540,20160002306,ZILMAR CANDIDO DE SANTANA JUNIOR,Vínculos Cosmológicos com dados de Quasares e ...,PIJ12992-2016,117607579,2016,5757556,PEDRO DA CUNHA FERREIRA,Iniciação Científica (IC),ECT,Cosmologia e gravitação,<NA>,NaN,2019-2020,2019-08-04,2020-03-03,4885,ESCOLA DE CIÊNCIAS E TECNOLOGIA - ECT,PENDENTE DE RELATORIO
40498,72529,200214205,ZORAIDE BEZERRA GOMES,Impactos da operação do Estado como empregador...,PVE1495-2007,1173409,2007,26268,ANDRE LUIS CABRAL DE LOURENCO,Iniciação Científica (IC),PIBIC,Estratégias para o Desenvolvimento Regional,42,Núcleo de Economia Aplicada e Quantitativa - NEAQ,2007-2008,2007-07-28,2008-03-10,160,DEPARTAMENTO DE ECONOMIA - DEPEC - DECO/CCSA,FINALIZADO
40499,538319,2009303264,ZUELITON DA SILVA ALMEIDA,CURVA DE RETENÇÃO DE ÁGUA EM NEOSSOLO QUARTZAR...,PVL6437-2010,50569684,2010,5753876,ERMELINDA MARIA MOTA OLIVEIRA,Iniciação Científica (IC),PIBIC-EM,Produção de Forragens,109,AGROECOLOGIA E DESENVOLVIMENTO SUSTENTÁVEL NO ...,Ensino Médio/Técnico 2010-2012,2010-10-01,2011-08-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
40500,536654,2010327302,ZUILA MOURA CAVALCANTI,AS PRÁTICAS INTEGRATIVAS EM SAÚDE E OS SERVIDO...,PID6336-2010,4918830,2010,5753559,MÉRCIA MARIA DE SANTI,Iniciação Científica (IC),PIBIC-EM,NENHUMA,<NA>,NaN,Ensino Médio/Técnico 2010-2012,2010-10-01,2011-12-14,205,ESCOLA DE SAÚDE - ES/UFRN,FINALIZADO


### Padronizar o nome das colunas para o mesmo definido na modelagem

In [13]:
df = df.rename(columns={'id_projeto_pesquisa': 'cd_projeto_pesquisa', 'orientador': 'nm_orientador', 'tipo_de_bolsa'
                        : 'tipo_bolsa', 'id_grupo_pesquisa': 'id_grupo', 'grupo_pesquisa': 'nm_grupo', 'inicio'
                        : 'inicio_pesquisa', 'fim': 'fim_pesquisa', 'unidade': 'nm_unidade'})

#### Dropar a linha caso tenha a metade ou mais nulo

In [14]:
df.dropna(axis=0, thresh=10, inplace=True)

## Transform - Tratamento dos Registros

### Vericar se contém caracteres especiais nos campos que não podem ter.

In [15]:
df.loc[df.discente.notnull() & (df['discente'].astype(str).str.replace(" ", "").str.isalnum() == False)]

,id_discente,matricula,discente,titulo,codigo_projeto,cd_projeto_pesquisa,ano,id_orientador,nm_orientador,categoria,tipo_bolsa,linha_pesquisa,id_grupo,nm_grupo,cota,inicio_pesquisa,fim_pesquisa,id_unidade,nm_unidade,status
6874,801323,20200003037,CARMEM JÚLIA SANT'ANNA DE OLIVEIRA,"Modalidades, lógicas sub-estruturais e a espec...",PIJ16135-2019,135002828,2019,5756674,CARLOS ALBERTO OLARTE VEGA,Iniciação Científica (IC),PIBIC CNPq,Lógica e Teoria de Concorrência,<NA>,NaN,2021-2022 (PIBIC),2021-09-01,2022-08-31,4885,ESCOLA DE CIÊNCIAS E TECNOLOGIA - ECT,FINALIZADO
6875,801323,20200003037,CARMEM JÚLIA SANT'ANNA DE OLIVEIRA,Teoria da prova em Lógica de Reescrita,PIJ16135-2019,135002828,2019,5756674,CARLOS ALBERTO OLARTE VEGA,Iniciação Científica (IC),PIBIC CNPq,Lógica e Teoria de Concorrência,<NA>,NaN,2020-2021,2021-03-15,2021-08-31,4885,ESCOLA DE CIÊNCIAS E TECNOLOGIA - ECT,FINALIZADO
7959,510606,200729993,CYNTHIA FIRMINO AIRES.,CARACTERIZAÇÃO FÍSICO-QUÍMICA DA ÁGUA MARINHA ...,PVA5092-2010,2361875,2010,5753068,MARIA CHRISTINA BARBOSA DE ARAUJO,Iniciação Científica (IC),PROPESQ,Oceanografia físico-química,9,BASE DE PESQUISA: ECOLOGIA E AQÜICULTURA DOS S...,2011,2011-01-05,2011-12-30,180,DEPARTAMENTO DE OCEANOGRAFIA E LIMNOLOGIA - DO...,FINALIZADO
8918,767920,20180138992,D'ÁVILA REGINA SILVA RODRIGUES,Plano 01 - Expectativas dos professores da UFR...,PVE13493-2016,117754916,2016,5756268,ANTONIO ALVES FILHO,Iniciação Científica (IC),VOLUNTÁRIO,Políticas e Gestão Públicas,43,GRUPO DE ESTUDOS E PESQUISAS EM GESTÃO INSTITU...,2017-2018,2018-03-09,2018-07-31,11612,DEPARTAMENTO DE ADMINISTRAÇÃO PUBLICA E GESTÃO...,FINALIZADO
9557,90655,200507015,DIEGO D'AVILA BESERRA,Apoio a coleta e processamento de dados sismicos,PVB1590-2009,12434319,2009,26059,HELENICE VITAL,Iniciação Científica (IC),PIBIC,sismica rasa,911656,Ciências do Mar e Ambientais,2009-2010,2010-06-01,2010-07-31,123,DEPARTAMENTO DE GEOLOGIA - GEO/CCET,FINALIZADO
12167,722276,20160126887,FELIPE GEORG'S COSTA DE OLIVEIRA,O uso do LED de alta potência na Odontologia c...,PID14135-2017,123501974,2017,5756311,SERGEI GODEIRO FERNANDES RABELO CALDAS,Iniciação Científica (IC),PIBIC UFRN,ESTUDO DAS TECNOLOGIAS APLICADAS À ODONTOLOGIA,<NA>,NaN,2019-2020,2020-01-26,2020-07-31,195,DEPARTAMENTO DE ODONTOLOGIA - DOD/CCS,FINALIZADO
12168,722276,20160126887,FELIPE GEORG'S COSTA DE OLIVEIRA,Qual o período ideal de indicação para interce...,PID17698-2020,141627967,2020,5756311,SERGEI GODEIRO FERNANDES RABELO CALDAS,Iniciação Científica (IC),PIBIC UFRN,"Epidemiologia, diagnóstico e tratamento integr...",<NA>,NaN,2020-2021,2020-08-09,2021-08-31,195,DEPARTAMENTO DE ODONTOLOGIA - DOD/CCS,FINALIZADO
12308,92363,200512242,FELIZARDO LIMA FERNANDES SOUSA D' ALMEIDA,Processo de coogeração de energia utilizando p...,PVG888-2008,5525012,2008,22939,ANDRES ORTIZ SALAZAR,Iniciação Científica (IC),PIBIC,Plasma;Eletronica de Potencia;;,18,CONTROLE E ACIONAMENTO DE SISTEMAS,2008-2009,2008-08-01,2009-07-31,1572,CT VICE-DIRETORIA - CT-VICEDIR,FINALIZADO
13503,805106,20200051900,GABRIELA D'JAH FERNANDES HOLANDA,GOVERNANÇA E POLÍTICAS PÚBLICAS NO CAMPO TURÍS...,PVE19007-2021,151176511,2021,5755548,WILKER RICARDO DE MENDONCA NOBREGA,Iniciação Científica (IC),PIBIC CNPq,"Turismo, Governança e Governabilidade",98742538,GEPPOT  GRUPO DE PESQUISA EM PLANEJAMENTO E O...,2022-2023 (PIBIC),2022-09-05,2023-08-31,5393,DEPARTAMENTO DE TURISMO - DETUR - DETUR,EM ANDAMENTO
14499,769949,20180138544,GIOVANA SOUZA D'OLERON BARRETO,IMPACTO DA CAPACITAÇÃO DA OXIMETRIA DE PULSO P...,PID19404-2021,156815368,2021,5753854,JONAS SAMI ALBUQUERQUE DE OLIVEIRA,Iniciação Científica (IC),VOLUNTÁRIO,"Teste do Coraçãozinho, Triagem Neonatal.",<NA>,NaN,2021-2022 (PIBIC),2022-03-09,2022-08-31,193,DEPARTAMENTO DE PEDIATRIA - DPEDI/CCS,PENDENTE DE RELATORIO


In [16]:
# Realizar o tratamento para os valores anteriores
df.discente = df.discente.str.replace('.', '').str.replace('-', '')

C:\Users\demik\AppData\Local\Temp\ipykernel_18504\758677198.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.discente = df.discente.str.replace('.', '').str.replace('-', '')


In [17]:
# Vericar se existem inconsistências no nome do nm_orientador

df.loc[df.nm_orientador.notnull() & (df['nm_orientador'].astype(str).str.replace(" ", "").str.isalnum() == False)]

,id_discente,matricula,discente,titulo,codigo_projeto,cd_projeto_pesquisa,ano,id_orientador,nm_orientador,categoria,tipo_bolsa,linha_pesquisa,id_grupo,nm_grupo,cota,inicio_pesquisa,fim_pesquisa,id_unidade,nm_unidade,status


In [18]:
# Mudar o limite de linha e caracteres nos campos para facilitar a visualização dos próximos campos
pd.options.display.max_colwidth = 200

In [19]:
# No caso do nm_grupo, não pode ter

df.loc[df.nm_grupo.notnull() & (df['nm_grupo'].astype(str).str.replace(" ", "").str.isalnum() == False)]

,id_discente,matricula,discente,titulo,codigo_projeto,cd_projeto_pesquisa,ano,id_orientador,nm_orientador,categoria,tipo_bolsa,linha_pesquisa,id_grupo,nm_grupo,cota,inicio_pesquisa,fim_pesquisa,id_unidade,nm_unidade,status
0,739553,20170093636,ABDA ROCHA PINHEIRO,Pesquisando práticas musicais no contexto de Natal-RN,PVH15882-2018,129546741,2018,5751926,JEAN JOUBERT FREITAS MENDES,Iniciação Científica (IC),VOLUNTÁRIO,Processos sociais e práticas musicais na contemporaneidade,135380677,GPMuC - Grupo de Pesquisa Música em Múltiplos Contextos,2019-2020,2019-08-05,2020-07-31,284,ESCOLA DE MÚSICA - EMUFRN,PENDENTE DE RELATORIO
11,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,Sistema de reprodução de espécies florestais da Caatinga,PVI11849-2012,107331336,2012,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2017-2018,2018-03-02,2018-07-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
12,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,IMPACTO DAS VARIAÇÕES CLIMÁTICAS NA ADEQUABILIDADE DE HABITAT DE Spondias tuberosa Arruda (ANACARDIACEAE),PVI16716-2019,135704984,2019,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2019-2020,2019-08-01,2020-07-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
13,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,Estrutura genética espacial de Syagrus cearensis: implicações para a conservação e coleta de sementes,PVI16716-2019,135704984,2019,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2021-2022 (PIBIC),2021-09-05,2022-02-23,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
14,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,Marcadores moleculares para estudos de diversidade genética da Syagrus cearensis Noblick,PVI16716-2019,135704984,2019,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2020-2021,2020-08-09,2021-08-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40477,785139,20190022453,ZAÚ JÚLIO ARAÚJO GALVÃO,"Estudo, seleção e aplicação de técnicas de aprendizado de máquina do tipo não supervisionado aos dados.",PVF16985-2019,135782281,2019,5753697,LUIZ PAULO DE ASSIS BARBOSA,Iniciação Científica (IC),VOLUNTÁRIO,Processamento de Sinais,50801320,"Comunicação, Processamento e Análise da Informação",2019-2020,2019-08-01,2020-03-28,9726,DEPARTAMENTO DE COMPUTAÇÃO E TECNOLOGIA - DCT,FINALIZADO
40482,789625,20190104100,ZELIA HELOISA SUASSUNA OLIVEIRA,Mapeamento das práticas de escrita realizadas por acadêmicos do Bacharelado em Tecnologia da Informação no ambiente digital (continuação),PVT15729-2018,129509299,2018,5755939,LUCELIO DANTAS DE AQUINO,Iniciação Científica (IC),PIBIC UFRN,Letramento digital: formação de professores e ensino e aprendizagem,152832931,"Grupo de Pesquisa em Linguagem, Ensino e Tecnologias Educacionais (LEnTE)",2019-2020,2019-11-01,2020-07-31,6069,INSTITUTO METROPOLE DIGITAL - IMD,PENDENTE DE RELATORIO
40490,545291,2011052223,ZILENE TAVARES DE CASTRO,A Literatura Comparada na América Latina e a literatura de Gabriel García Márquez,PVC5349-2011,85036943,2011,5752297,GERARDO ANDRES GODOY FAJARDO,Iniciação Científica (IC),VOLUNTÁRIO,Literatura Comparada,2228952,Grupo de Pesquisa Ponte Literária Hispano-brasileira,2011-2012,2012-04-12,2012-07-31,5206,DEPARTAMENTO DE LINGUAS E LITERATURAS ESTRANGEIRAS MODERNAS - DLLEM,FINALIZADO
40498,72529,200214205,ZORAIDE BEZERRA GOMES,"Impactos da operação do Estado como empregador de última instância sobre a economias brasileira, nordestina e potiguar.",PVE1495-2007,1173409,2007,26268,ANDRE LUIS CABRAL DE LOURENCO,Iniciação Científica (IC),PIBIC,Estratégias para o De

In [20]:
# Mudar o limite de linha e caracteres nos campos para facilitar a visualização dos próximos 
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 50

### Tratar o nm_grupo, encontrado "." em alguns registros, devem ser compreendidos e retirados caso necessário

In [21]:
# No caso do nm_grupo

df.loc[df.nm_grupo.notnull() & (df['nm_grupo'].str.contains(".", regex=False))]

,id_discente,matricula,discente,titulo,codigo_projeto,cd_projeto_pesquisa,ano,id_orientador,nm_orientador,categoria,tipo_bolsa,linha_pesquisa,id_grupo,nm_grupo,cota,inicio_pesquisa,fim_pesquisa,id_unidade,nm_unidade,status
11,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,Sistema de reprodução de espécies florestais da Caatinga,PVI11849-2012,107331336,2012,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2017-2018,2018-03-02,2018-07-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
12,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,IMPACTO DAS VARIAÇÕES CLIMÁTICAS NA ADEQUABILIDADE DE HABITAT DE Spondias tuberosa Arruda (ANACARDIACEAE),PVI16716-2019,135704984,2019,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2019-2020,2019-08-01,2020-07-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
13,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,Estrutura genética espacial de Syagrus cearensis: implicações para a conservação e coleta de sementes,PVI16716-2019,135704984,2019,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2021-2022 (PIBIC),2021-09-05,2022-02-23,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
14,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,Marcadores moleculares para estudos de diversidade genética da Syagrus cearensis Noblick,PVI16716-2019,135704984,2019,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2020-2021,2020-08-09,2021-08-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
15,743378,20170125373,ABIDÃ GÊNESIS DA SILVA NEVES,Impacto das mudanças climáticas na adequabilidade ambiental da palmeira carnaúba (Copernicia prunifera),PVI11849-2012,107331336,2012,5752710,FABIO DE ALMEIDA VIEIRA,Iniciação Científica (IC),PIBIC CNPq,Genética e melhoramento florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2018-2019,2018-08-02,2019-07-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40153,822610,20210013304,YAGO TOMAZ VIEIRA DA SILVA,Identificação de vírus que potencialmente afetam a fertilidade em amostras de sêmen - continuação,PIA15837-2018,129541637,2018,5754348,DANIEL CARLOS FERREIRA LANZA,Iniciação Científica (IC),PIBIC UFRN,Desenvolvimento de ferramentas moleculares para identificação de patógenos de importância médica.,135476589,"Biologia molecular aplicada ao estudo, identificação e controle de patógenos.",2021-2022 (PIBIC),2022-03-31,2022-05-04,182,DEPARTAMENTO DE BIOQUÍMICA - DBQ/CB,FINALIZADO
40154,822610,20210013304,YAGO TOMAZ VIEIRA DA SILVA,Identificação dos vírus HPV e HSV em amostras de sêmen,PVA19535-2022,159535401,2022,5754348,DANIEL CARLOS FERREIRA LANZA,Iniciação Científica (IC),PIBIC CNPq,Desenvolvimento de ferramentas moleculares para identificação de patógenos de importância médica.,135476589,"Biologia molecular aplicada ao estudo, identificação e controle de patógenos.",2022-2023 (Ebserh),2022-05-04,2023-04-30,182,DEPARTAMENTO DE BIOQUÍMICA - DBQ/CB,EM ANDAMENTO
40230,601797,2013054334,YASMIM BORGES CÂMARA,AVALIAÇÃO DO CRESCIMENTO DA JUREMA-PRETA (Mimosa tenuiflora) EM PLANTIOS HOMOGÊNEOS COM 5 ANOS DE IDADE,PVI14286-2017,123608105,2017,5753897,JULIANA LORENSI DO CANTO,Iniciação Científica (IC),VOLUNTÁRIO,Silvicultura e Manejo Florestal,4878102,Florestas. Bioenergia e Meio Ambiente,2017-2018,2017-08-08,2018-01-24,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
40239,784564,20190012260,YASMIM KETCHURE GONZAGA DA SILVA,ASPECTOS PESSOAIS E CLÍNICOS DOS ALUNOS DO 3º PERÍODO DO CURSO DE MEDICINA ANTES DA PANDEMIA DA COVID-19,PVA18969-2021,151145256,2021,2262005,NAIANNE KELLY CLEBIS,Iniciação Científica (IC),VOLUN

In [22]:
df.loc[df.nm_grupo.notnull() & df.nm_grupo.str.endswith('.'), 'nm_grupo']

1450             Biologia molecular aplicada ao estudo, identificação e controle de patógenos.
2003           CUIDADOS COM A SAÚDE FEMININA E MATERNO-INFANTIL NA MATERNIDADE JANUÁRIO CICCO.
2633                                             Neuroengenharia e Engenharia de Reabilitação.
4955                                                                                         .
4956                                                                                         .
                                                 ...                                          
39490                                        Estudo de novas técnicas de fiação de nanofibras.
39491                                        Estudo de novas técnicas de fiação de nanofibras.
39569    Grupo Eduardo Galeano de Estudos e Pesquisas Latino-Americanas em Comunicação Social.
40153            Biologia molecular aplicada ao estudo, identificação e controle de patógenos.
40154            Biologia molecular aplicada ao es

#### Alguns registros, contém "." no final devem ser substituido por nada. 

In [23]:
selecao = (df.nm_grupo.notnull()) & (df.nm_grupo.str.endswith('.'))
df.loc[selecao, 'nm_grupo'] = df.loc[selecao, 'nm_grupo'].str.replace('.', '', regex=False)
df

,id_discente,matricula,discente,titulo,codigo_projeto,cd_projeto_pesquisa,ano,id_orientador,nm_orientador,categoria,tipo_bolsa,linha_pesquisa,id_grupo,nm_grupo,cota,inicio_pesquisa,fim_pesquisa,id_unidade,nm_unidade,status
0,739553,20170093636,ABDA ROCHA PINHEIRO,Pesquisando práticas musicais no contexto de Natal-RN,PVH15882-2018,129546741,2018,5751926,JEAN JOUBERT FREITAS MENDES,Iniciação Científica (IC),VOLUNTÁRIO,Processos sociais e práticas musicais na contemporaneidade,135380677,GPMuC - Grupo de Pesquisa Música em Múltiplos Contextos,2019-2020,2019-08-05,2020-07-31,284,ESCOLA DE MÚSICA - EMUFRN,PENDENTE DE RELATORIO
1,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avançada por fotocatálise heterogênea destinada ao tratamento de correntes aquosas com hidrocarbonetos,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Tecnológica (IT),PIBITI UFRN,Fotoquímica e Equilíbrio de Fases,77,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020-08-10,2020-09-03,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
2,716941,2016092160,ABDÊNIO VICTOR PEREIRA DA SILVA,Nanopartículas magnéticas para oxidação avançada por fotocatálise heterogênea destinada ao tratamento de correntes aquosas com hidrocarbonetos,PVG13574-2016,117767764,2016,22926,OSVALDO CHIAVONE FILHO,Iniciação Científica (IC),PIBIC AF CNPq,Fotoquímica e Equilíbrio de Fases,77,TECNOLOGIA DE TENSOATIVOS,2020-2021,2020-09-03,2021-08-31,57,DEPARTAMENTO DE ENGENHARIA QUIMICA - DEQ/CT,FINALIZADO
3,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estruturação da sua rede local e mundial: aspectos necessários a uma competitividade sustentável a médio e longo prazo,PVE334-2007,1179265,2007,64653,MARIANA BALDI,Iniciação Científica (IC),PIBIC,Organizações e comportamento organizacional;;;,38,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E POLÍTICAS DE GESTÃO,2007-2008,2007-08-01,2008-07-31,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
4,88234,200504593,ABDON SILVA RIBEIRO DA CUNHA,O agronegócio da carcinicultura no RN e a estruturação da sua rede local e mundial: aspectos necessários a uma competitividade sustentável a médio e longo prazo,PVE334-2006,41258,2006,64653,MARIANA BALDI,Iniciação Científica (IC),PROPESQ,Organizações e comportamento organizacional;;;,38,BASE DE ESTUDOS E PESQUISAS EM ESTRATÉGIAS E POLÍTICAS DE GESTÃO,2006-2007,2006-08-01,2007-07-31,443,CENTRO DE CIÊNCIAS SOCIAIS APLICADAS - CCSA,FINALIZADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40497,711540,20160002306,ZILMAR CANDIDO DE SANTANA JUNIOR,Vínculos Cosmológicos com dados de Quasares e Parâmetro de Hubble,PIJ12992-2016,117607579,2016,5757556,PEDRO DA CUNHA FERREIRA,Iniciação Científica (IC),ECT,Cosmologia e gravitação,<NA>,NaN,2019-2020,2019-08-04,2020-03-03,4885,ESCOLA DE CIÊNCIAS E TECNOLOGIA - ECT,PENDENTE DE RELATORIO
40498,72529,200214205,ZORAIDE BEZERRA GOMES,"Impactos da operação do Estado como empregador de última instância sobre a economias brasileira, nordestina e potiguar.",PVE1495-2007,1173409,2007,26268,ANDRE LUIS CABRAL DE LOURENCO,Iniciação Científica (IC),PIBIC,Estratégias para o Desenvolvimento Regional,42,Núcleo de Economia Aplicada e Quantitativa - NEAQ,2007-2008,2007-07-28,2008-03-10,160,DEPARTAMENTO DE ECONOMIA - DEPEC - DECO/CCSA,FINALIZADO
40499,538319,2009303264,ZUELITON DA SILVA ALMEIDA,CURVA DE RETENÇÃO DE ÁGUA EM NEOSSOLO QUARTZARÊNICO SOB PLANTIO DE PALMA FORRAGEIRA,PVL6437-2010,50569684,2010,5753876,ERMELINDA MARIA MOTA OLIVEIRA,Iniciação Científica (IC),PIBIC-EM,Produção de Forragens,109,AGROECOLOGIA E DESENVOLVIMENTO SUSTENTÁVEL NO SEMIÁRIDO (CAATINGUEIROS),Ensino Médio/Técnico 2010-2012,2010-10-01,2011-08-31,351,ESCOLA AGRÍCOLA DE JUNDIAÍ - EAJ,FINALIZADO
40500,536654,2010327302,ZUILA MOURA CAVALCANTI,AS PRÁTICAS INTEGRATIVAS EM SAÚDE E OS SERVIDORES DA ESCOLA DE ENFERMAGEM DA UFRN,PID6336-2010,4918830,2010,5753559,MÉRCIA MARIA DE SANTI,In

In [24]:
df1.head(10)

0    135380677.0
1           77.0
2           77.0
3           38.0
4           38.0
5           43.0
6            nan
7           38.0
8           16.0
9          112.0
Name: id_grupo_pesquisa, dtype: object

#### Na última instrução, o registro de ID 133488647, ficou com nada, pois só tinha um ponto. Conforme a regra de negócio 17, não é admitido nulo neste campo. Deve ser substituido por "Não informado"

In [25]:
df.loc[df.id_grupo == '133488647', 'nm_grupo'] = 'Não Informado'

#### Os grupos de pesquisa de ID 4878102 e 13087450, tem o ponto como vírgula

In [26]:
selecao = (df.id_grupo.notnull()) & (df.id_grupo.str.match('(4878102)|(13087450)'))
df.loc[selecao, 'nm_grupo'] = df.loc[selecao, 'nm_grupo'].str.replace('.', ',', regex=False)

In [27]:
df[df.id_grupo == '13087450']

,id_discente,matricula,discente,titulo,codigo_projeto,cd_projeto_pesquisa,ano,id_orientador,nm_orientador,categoria,tipo_bolsa,linha_pesquisa,id_grupo,nm_grupo,cota,inicio_pesquisa,fim_pesquisa,id_unidade,nm_unidade,status
718,742778,20170120081,ALEILSON ABNER CAMARA DA SILVA,ANÁLISE HISTOMORFOMÉTRICA DA ARTÉRIA AORTA EM ANIMAIS DIABÉTICOS SUBMETIDOS À OXIGENOTERAPIA HIPERBÁRICA,PVA16446-2019,135648178,2019,5752233,BENTO JOAO DA GRACA AZEVEDO ABREU,Iniciação Científica (IC),PIBIC UFRN,Repercussões morfológicas em modelos experimentais de Diabetes Mellitus,13087450,"PLASTICIDADE MORFOFUNCIONAL DOS SISTEMAS ORGÂNICOS, MICROSCOPIA CELULAR E TECIDUAL",2021-2022 (PIBIC),2021-09-07,2022-08-19,177,DEPARTAMENTO DE MORFOLOGIA - MOR/CB,FINALIZADO
719,742778,20170120081,ALEILSON ABNER CAMARA DA SILVA,AVALIAÇÃO DO TRATAMENTO POR OXIGENOTERAPIA HIPERBÁRICA NO REMODELAMENTO CARDÍACO EM RATOS DIABÉTICOS,PVA16446-2019,135648178,2019,5752233,BENTO JOAO DA GRACA AZEVEDO ABREU,Iniciação Científica (IC),PIBIC UFRN,Repercussões morfológicas em modelos experimentais de Diabetes Mellitus,13087450,"PLASTICIDADE MORFOFUNCIONAL DOS SISTEMAS ORGÂNICOS, MICROSCOPIA CELULAR E TECIDUAL",2020-2021,2020-08-09,2021-08-31,177,DEPARTAMENTO DE MORFOLOGIA - MOR/CB,FINALIZADO
720,742778,20170120081,ALEILSON ABNER CAMARA DA SILVA,ANÁLISE GEOMÉTRICA E HISTOMORFOMÉTRICA DO REMODELAMENTO CARDÍACO EM ANIMAIS DIABÉTICOS SUBMETIDOS À OXIGENOTERAPIA HIPERBÁRICA,PVA14422-2017,123590581,2017,5752233,BENTO JOAO DA GRACA AZEVEDO ABREU,Iniciação Científica (IC),PIBIC UFRN,Morfofisiologia dos sistemas orgânicos,13087450,"PLASTICIDADE MORFOFUNCIONAL DOS SISTEMAS ORGÂNICOS, MICROSCOPIA CELULAR E TECIDUAL",2019-2020,2019-08-05,2020-07-31,177,DEPARTAMENTO DE MORFOLOGIA - MOR/CB,FINALIZADO
909,557900,2012913022,ALEXANDRE JOSÉ DE LIMA SALES JÚNIOR,CARACTERIZAÇÃO HISTOLÓGICA DO TRATO DIGESTORIO DE Cichlasoma orientale (OSTEICHTHYES: CICHLIDAE),PVA10705-2014,100952926,2014,2305973,NAISANDRA BEZERRA DA SILVA FARIAS,Iniciação Científica (IC),PROPESQ,Morfofisiologia comparada,13087450,"PLASTICIDADE MORFOFUNCIONAL DOS SISTEMAS ORGÂNICOS, MICROSCOPIA CELULAR E TECIDUAL",2014,2014-02-11,2015-03-17,177,DEPARTAMENTO DE MORFOLOGIA - MOR/CB,FINALIZADO
953,741834,20170140979,ALEXANDY MICHEL DANTAS SANTOS,Efeitos da oxigenoterapia hiperbárica em alterações morfológicas resultantes de nefropatia em ratos diabéticos.,PVA14229-2017,123541462,2017,2305973,NAISANDRA BEZERRA DA SILVA FARIAS,Iniciação Científica (IC),PIBIC UFRN,Morfofisiologia dos sistemas orgânicos,13087450,"PLASTICIDADE MORFOFUNCIONAL DOS SISTEMAS ORGÂNICOS, MICROSCOPIA CELULAR E TECIDUAL",2019-2020,2019-08-04,2020-07-31,177,DEPARTAMENTO DE MORFOLOGIA - MOR/CB,FINALIZADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39036,621472,2013083416,VILESSA LÍLIAN DE ARAÚJO GOMES,Morfometria testicular de camundongos com distrofia muscular de Duchenne tratados com ácido ascórbico,PVA10507-2014,100785187,2014,24706,CARLOS EDUARDO BEZERRA DE MOURA,Iniciação Científica (IC),PROPESQ,"Biologia Celular. Caracterização Estrutural e Ultraestrutural, Adesão, Proliferação, Diferenciação e Morte Celular",13087450,"PLASTICIDADE MORFOFUNCIONAL DOS SISTEMAS ORGÂNICOS, MICROSCOPIA CELULAR E TECIDUAL",2014,2014-02-10,2014-02-19,177,DEPARTAMENTO DE MORFOLOGIA - MOR/CB,PENDENTE DE RELATORIO
39067,522846,2009048113,VINÍCIUS CAMPELO SOEIRO,EFEITO DA SUPERFÍCIE DE TITÂNIO NITRETADA SOBRE A EXPRESSÃO DE INTEGRINAS EM CÉLULAS-TRONCO MESENQUIMAIS DA VEIA UMBILICAL,PVA2870-2009,11288155,2009,24706,CARLOS EDUARDO BEZERRA DE MOURA,Iniciação Científica (IC),PIBIC,Biologia do Desenvolvimento e Tecidual,13087450,"PLASTICIDADE MORFOFUNCIONAL DOS SISTEMAS ORGÂNICOS, MICROSCOPIA CELULAR E TECIDUAL",2009-2010,2009-08-01,2010-07-31,177,DEPARTAMENTO DE MORFOLOGIA - MOR/CB,FINALIZADO
39927,695914,2015078685,WIGNA PATRICIA VERAS DA SILVA,Avaliação do tratamento homeopático a base de Rhus toxidendron e Rhus graveolens na morfologia do tecido rena

In [28]:
# Verificar as alterações, deve aparecer apenas um grupo

selecao = df.nm_grupo.notnull() & df['nm_grupo'].str.contains(".", regex=False)
df.loc[selecao, ['id_grupo', 'nm_grupo']].drop_duplicates()

,id_grupo,nm_grupo
1073,129833868,Grupo de Pesquisa em Indústria 4.0


### Verificar se existem caracteres não convertidos

In [29]:
def col_caractere():
    col_list = []
    for col in df.columns: 
        try:
            mask = df[col].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
            if mask.any():
                col_list.append(col)
            else:
                continue
        except AttributeError:
            continue
    return col_list

In [30]:
col_caractere()

['titulo', 'linha_pesquisa', 'nm_grupo']

#### Será necessário entender quais caracteres são cada um e em cada coluna que tem

In [31]:
mask = df['titulo'].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
df.loc[mask, 'titulo'].drop_duplicates()

50                                                          Avaliação da qualidade de vida e resistência muscular e sua relação com o Índice Tornozelo  Braço (ITB) em portadores de doença arterial periférica
70                                                                           VULNERABILIDADE SOCIOAMBIENTAL, MUDANÇAS CLIMÁTICAS E ENERGIAS RENOVÁVEIS: ATUALIZAÇÃO BIBLIOGRÁFICA, TENDÊNCIAS E ESTUDOS  FASE 1
83                                                                  CICLO SONO VIGILIA: INTERVENÇÃO COMPORTAMENTAL EM ESTUDANTES DO 1° ANO DO CURSO DE PSICOLOGIA DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE
206                                                                              Tradução e Feminismo: traduções do conto A respectable woman, de Kate Chopin, como interpretação de obras de autoria feminina
207                                                                                                           Os psicólogos no trabalho com as populações de baixa r

In [32]:
mask = df['linha_pesquisa'].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
df.loc[mask, 'linha_pesquisa'].drop_duplicates()

596                                                                                                                                       Diagnóstico e epidemiologia de doenças infecciosas e parasitárias
778                                                                                                                                                       Linha 1  Enfermagem, educação, saúde e cidadania;
1756                                                                                             Avaliação do desenvolvimento motor típico e atípico e intervenção nas alterações neuromotoras da infância
3100     \tEletrônica de Potência \tAcionamento de Máquinas Elétricas  \tConversores estáticos  \tQualidade de energia  \tGeração Distribuída \tControladores analógicos e digitais \tMicroeletrônica
3955                                                                                                                                                                Linha 3  Ciênci

In [33]:
mask = df['nm_grupo'].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
df.loc[mask, 'nm_grupo'].drop_duplicates()

38      Grupo de Estudos Pol­ítica, Produção de Subjetividades e Práticas de Resistência
145                  GEPPOT  GRUPO DE PESQUISA EM PLANEJAMENTO E ORGANIZAÇÃO DO TURISMO
1943                              Audição e Equilíbrio:  Avaliação e Reabilitação  UFRN
2815      Grupo de Pesquisa em Educação e Trabalho Interprofissional em Saúde  GRUPETIS
5918        GRUPO DE ESTUDOS E PESQUISA EM PSICOBIOLOGIA DO EXERCÍCIO E ESPORTE  GEPPEE
Name: nm_grupo, dtype: object

#### 1 - Todos os campo em que o caractere está entre dois espaços deveria ser o meio hífen, porém poderá ser transformado em hífen. 

#### 2 - Se estiverem com o \t na frente, deve ser substituido por "- "

#### Percorrerá todas as colunas que tem o caractere.

In [34]:
regex = ['[\s][^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\s]', '[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\t]']

mask = df['nm_grupo'].fillna('').str.contains(r'{}'.format(regex[1]))
df.loc[mask, 'nm_grupo'].drop_duplicates()

Series([], Name: nm_grupo, dtype: object)

In [35]:
cols = col_caractere()
regex = ['[\s][^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\s]', '[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\t]']

for i in cols:
    df[i] = df[i].str.replace(regex[0], ' - ',  regex=True).str.replace(regex[1], '- ', regex=True)

#### Restaram apenas os caracteres que ficam no começo e fim de cada texto, esses serão apagados conforme a RN 20